In [11]:
import json
import vk_api
import os
from pathlib import Path
import time
from urllib.parse import urlparse
from vk_api.vk_api import VkApi, VkApiMethod
from typing import TextIO
from tqdm import tqdm
from dotenv import load_dotenv


In [12]:
BASE = Path().cwd()
MENO_RESOURCES = BASE.joinpath("resources")
MENO_DATABASE = BASE.joinpath("meno_database")

Задаём конфигурацию

In [20]:
load_dotenv()

# Сервисный ключ доступа
VK_SERVICE_TOKEN = os.getenv("VK_SERVICE_TOKEN")
if (VK_SERVICE_TOKEN is None):
    raise ValueError("❌ В .env файле не задан VK_SERVICE_TOKEN")
  
# Количество постов, которое необходимо скачать
POSTS_PER_GROUP = 100

# Имя входного файла
INPUT_FILE = MENO_RESOURCES.joinpath("vk_urls.json")

# Файл для сохранения результатов
OUTPUT = MENO_DATABASE.joinpath("vk_posts.jsonl")

Дополнительные переменные

In [14]:
# Количество постов для скачивания (максимум 100 за один запрос)
POSTS_PER_REQUEST = 100

Helper-функции

In [15]:
def get_group_domain(url):
    """Извлекает domain группы из ссылки"""
    return urlparse(url).path.strip("/")

In [18]:
def get_posts(vk: VkApiMethod, domain: str, count: int, offset: int):
    response = vk.wall.get(domain=domain, count=count, offset=offset)
    posts = response.get('items', [])
    return posts

def to_output_dict(post: dict, name: str, link: str) -> dict:
    result = dict();
    result['url'] = link
    result['name'] = name
    result['content'] = post.get('text', ' ')
    return result

def collect_data(vk: VkApiMethod, domain: str, title: str, link: str, posts_number: int, posts_per_request: int, out: TextIO):
    if (posts_per_request < 1):
        print("ERROR: posts per group < 1")

    requests_number = posts_number // posts_per_request
    last_request = posts_number % posts_per_request
    
    requests = [(posts_per_request, posts_per_request * i) for i in range(requests_number)]
    if (last_request > 0):
        requests.append((last_request, posts_per_request * requests_number))
    
    for count, offset in tqdm(requests):
        posts = get_posts(vk, domain, count=count, offset=offset)
        
        for post in posts:
            # Добавляем мета-информацию, чтобы знать, откуда пост
            out_post = to_output_dict(post, title, link)
            json_line = json.dumps(out_post, ensure_ascii=False)
            
            out.write(json_line + '\n')
        
        print(f"✅ Cохранено {count + offset} постов")
        
        # Небольшая пауза, чтобы ВК не заблокировал за частые запросы
        time.sleep(0.3)

In [21]:
def main():
    # 1. Авторизация
    try:
        vk_session = vk_api.VkApi(token=VK_SERVICE_TOKEN)
        vk: VkApiMethod = vk_session.get_api()
    except Exception as e:
        print(f"❌ Ошибка авторизации: {e}")
        return

    # 2. Чтение списка групп
    try:
        with open(INPUT_FILE, 'r', encoding='utf-8') as f:
            groups_dict = json.load(f)
    except FileNotFoundError:
        print(f"❌ Файл {INPUT_FILE} не найден.")
        return

    print(f"Найдено групп: {len(groups_dict)}. Начинаем сбор в {OUTPUT}...")

    # Открываем файл один раз на запись (режим 'w' очистит файл при запуске)
    # Если хотите дописывать к существующему, используйте режим 'a'
    with open(OUTPUT, 'w', encoding='utf-8') as f_out:
        
        for title, link in groups_dict.items():
            domain = get_group_domain(link)
            print(f"Downloading: {title}...", end=" ")
            
            try:
                collect_data(vk, domain, title, link, POSTS_PER_GROUP, POSTS_PER_REQUEST, f_out)

            except vk_api.exceptions.ApiError as e:
                print(f"\n⚠️ Ошибка API ({title}): {e}")
            except Exception as e:
                print(f"\n⚠️ Ошибка ({title}): {e}")

    print(f"\n🎉 Готово! Все данные сохранены в {OUTPUT}")

if __name__ == "__main__":
    main()

Найдено групп: 1. Начинаем сбор в /home/thinkingduck/Life/Menon/Scrapper/meno_database/vk_posts.jsonl...
Downloading: Новосибирский государственный университет | НГУ... 

  0%|          | 0/1 [00:00<?, ?it/s]

✅ Cохранено 100 постов


100%|██████████| 1/1 [00:01<00:00,  1.61s/it]


🎉 Готово! Все данные сохранены в /home/thinkingduck/Life/Menon/Scrapper/meno_database/vk_posts.jsonl
